In [1]:
import numpy as np
import pandas as pd
import time
import sys
import random
from helper_functions_assignment2 import*

In [2]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
#problem_instance = "heur002_n_100_m_3274" #"heur002_n_100_m_3274" #"heur040_n_300_m_13358"
problem_instance = "heur002_n_100_m_3274"#"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [3]:
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
print(s)
print(n)
print(m)

2
100
3274


In [4]:
plex_assignment = np.random.choice(3, n) #np.range(n) of size n (second is the size)
solution = assignment_to_solution(plex_assignment, node_impact_orig, 
                                    node_degree_orig, edge_weights, edge_assignment_orig, s)

old = solution.edge_assignment

print(np.where(old == 0))

plex_assignment, edge_assignment, removed = remove_random_node(plex_assignment, solution.edge_assignment, n, node_impact_orig, node_degree_orig, edge_weights)

with np.printoptions(threshold=np.inf):
    print(removed)
    print(np.where(edge_assignment == 0))

(array([   1,    2,    3, ..., 4944, 4947, 4948], dtype=int64),)
node degrees [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[29]
(array([   1,    2,    3,    4,    6,    7,    8,    9,   10,   13,   14,
         15,   23,   24,   25,   28,   30,   31,   32,   34,   35,   37,
         38,   39,   43,   45,   46,   47,   48,   49,   52,   53,   55,
         57,   59,   60,   62,   63,   64,   65,   69,   71,   72,   73,
         74,   75,   76,   77,   79,   80,   81,   82,   84,   86,   87,
         88,   89,   90,   91,   92,   93,   94,   96,   99,  100,  101,
        102,  104,  105,  106,  107,  108,  111,  112,  113,  121,  122,
        123,  126,  128,  129,  130,  132,  133,  135,  136,  137,  141,
        143,  144,  145,  146,  147,  150,  151,  152,  153,  155,  157,
        158,  160,  161,  163,  167,  169,  170,  171,  172,  173,  174,
        175,  177,  178,  179,  180,  182,  184,  185,  186,  187,  188,
        189,  190,  191,  192,  194,  2

In [5]:
random.seed(4)
destroy_methods = [remove_random_node, remove_highest_cost_node, remove_most_edges_node, remove_smallest_splex, 
                   remove_largest_splex]
repair_methods = [add_to_smallest_splex, add_to_largest_splex, add_new_splex, add_to_random_splex]

destroy_weights = np.ones(len(destroy_methods))
repair_weights = np.ones(len(repair_methods))

# Parameters:
min_weight = 0.05
reaction_factor = 0.1
T_init = 10000
use_metropolis = True
cooling = 0.75
iterations_per_phase = 10
metropolis_success_value = 0.5 # Metropolis = partial success?

def update_weight(old_weight, num_applications, num_successes):
    if num_applications == 0:
        return old_weight
    new_weight = old_weight * (1-reaction_factor) + reaction_factor * (num_successes / num_applications)
    return max(new_weight, min_weight) # Make sure weight never equals 0

# get all infos from instance
node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
node_impact = node_impact_orig.copy()
node_degree = node_degree_orig.copy()
edge_assignment = edge_assignment_orig.copy()

plex_assignment = np.random.choice(3, n) #np.range(n) of size n (second is the size)
print("initial plex assignment", plex_assignment)

# correct node and edge infos to make a valid plex
repair_solution(node_impact, node_degree, plex_assignment, edge_weights, edge_assignment, s)

print(edge_assignment_orig)
print("node degree", node_degree)

T = T_init
#edge_assignment = solution.edge_assignment
print(edge_assignment)
current_score = sum(node_impact)/2

for i in range(100):
    destroy_prob = destroy_weights/destroy_weights.sum()
    repair_prob = repair_weights/repair_weights.sum()

    print(destroy_prob)
    print(repair_prob)

    destroy_applications = np.zeros(len(destroy_methods))
    destroy_successes = np.zeros(len(destroy_methods))
    repair_applications = np.zeros(len(repair_methods))
    repair_successes = np.zeros(len(repair_methods))

    for j in range(iterations_per_phase):
        destroy_idx = np.random.choice(len(destroy_methods), p=destroy_prob)
        repair_idx = np.random.choice(len(repair_methods), p=repair_prob)
        
        destroy_applications[destroy_idx] += 1
        repair_applications[repair_idx] += 1

        plex_assignment_new = plex_assignment.copy()
        edge_assignment_new = edge_assignment.copy()
        node_impact_new = node_impact.copy()
        node_degree_new = node_degree.copy()
        
        print(destroy_methods[destroy_idx])
        _, _, removed_nodes = destroy_methods[destroy_idx](plex_assignment_new, edge_assignment_new, n, node_impact_new, node_degree_new, edge_weights)
        print(repair_methods[repair_idx])
        repaired = repair_methods[repair_idx](plex_assignment_new, edge_assignment_new, n, removed_nodes, node_impact_new, node_degree_new, edge_weights, s)
        
        new_score = sum(node_impact_new)/2
        accept_solution = False
        if new_score < current_score:
            accept_solution = True
            destroy_successes[destroy_idx] += 1
            repair_successes[repair_idx] += 1
        elif use_metropolis:
            metropolis = math.exp(-abs(new_score-current_score)/T)
            P = random.uniform(0,1)
            if P < metropolis: # accept solution anyway
                accept_solution = True
                destroy_successes[destroy_idx] += metropolis_success_value
                repair_successes[repair_idx] += metropolis_success_value

        if accept_solution:
            current_score = new_score
            plex_assignment = plex_assignment_new
            edge_assignment = edge_assignment_new
            node_impact = node_impact_new
            node_degree = node_degree_new          

    for j in range(0, len(destroy_methods)):
        destroy_weights[j] = update_weight(destroy_weights[j], destroy_applications[j], destroy_successes[j])
        
    for j in range(0, len(repair_methods)):
        repair_weights[j] = update_weight(repair_weights[j], repair_applications[j], repair_successes[j])

    if use_metropolis:
        T *= cooling
    
    print("iter")

initial plex assignment [0 1 1 0 0 0 2 2 0 1 2 1 1 1 2 1 2 1 2 1 1 1 0 1 1 0 2 2 0 0 2 2 2 0 1 0 2
 2 2 1 2 1 1 2 1 1 1 0 2 1 1 0 2 1 2 2 1 2 1 0 2 2 0 2 0 2 1 0 2 1 0 1 0 0
 2 0 1 1 0 1 0 0 0 2 0 0 0 0 2 0 1 2 2 0 1 0 1 2 1 1]
[0 0 0 ... 0 0 0]
node degree [31 34 34 31 30 30 30 30 31 34 30 34 34 34 30 34 30 34 31 35 34 35 30 34
 35 31 31 31 30 30 30 30 31 30 35 30 30 30 30 35 31 35 34 30 35 35 34 31
 31 34 34 31 31 34 30 30 35 30 34 30 30 30 30 30 31 31 34 31 31 35 31 35
 31 31 30 31 35 35 30 35 31 30 31 31 31 30 30 31 31 31 34 31 30 30 35 31
 34 30 34 35]
[0 0 1 ... 0 0 1]
[0.2 0.2 0.2 0.2 0.2]
[0.25 0.25 0.25 0.25]
<function remove_smallest_splex at 0x0000029E22EFCCC0>
nodes in plex [ 1  4  5  6  9 23 26 29 30 34 36 48 52 60 63 65 68 71 73 74 76 79 81 82
 83 85 86 87 88 90 94 96]
node degrees [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
<function add_to_largest_splex at 0x0000029E22EFCEA0>
<function remove_most_edges_node at 0x0000029E22EFCB80>
node degrees [65 

ValueError: kth(=91) out of bounds (1)

In [ ]:
get_edge_nodes(43, n)

(8, 10)